In [1]:
import gc   # For manual garbage collection.
import numpy as np
import pandas as pd
import scipy.stats as stats
from matplotlib import pyplot as plt
from numba import njit

experiment = 'Exp1'

# Import and select data.
all_data = pd.read_csv('total_data.csv',
    usecols = ['gps.lat', 'gps.lon', 'altitudeRelative', 'Concentration', 'Experiment'],
)
all_data = all_data[all_data['Experiment'] == experiment]
all_data = all_data.drop(columns = ['Experiment'])

# Import and select metadata.
experiment_metadata = pd.read_csv('data_summary.csv',
    usecols = ['Experiment', 'Wind_Dir', 'WindSpeed', 'boat.lat', 'boat.lon']
)
experiment_metadata = experiment_metadata[experiment_metadata['Experiment'] == experiment]
wind_dir = experiment_metadata['Wind_Dir'].values[0]
wind_speed = experiment_metadata['WindSpeed'].values[0]

# Converting lat and lon to distances from boat in downwind and crosswind directions.
all_data['dist_lat'] = (all_data['gps.lat'] - experiment_metadata['boat.lat'].values[0]) * 111000
all_data['dist_lon'] = (all_data['gps.lon'] - experiment_metadata['boat.lon'].values[0]) * 111000
all_data['x'] = all_data['dist_lon'] * np.cos(270 - wind_dir) + all_data['dist_lat'] * np.sin(270 - wind_dir)
all_data['y'] = all_data['dist_lon'] * np.cos(360 - wind_dir) + all_data['dist_lat'] * np.sin(360 - wind_dir)
all_data['z'] = all_data['altitudeRelative']

# Modify concentrations.
all_data['Concentration'] = all_data['Concentration'] * 100 ** 3

# Split data, 80% for training and 20% for testing, shuffling rows first.
all_data = all_data.drop(columns = ['altitudeRelative', 'dist_lat', 'dist_lon', 'gps.lat', 'gps.lon'])
all_data = np.asarray(all_data)   # Prepare for Numba.
np.random.seed(1)                 # Ensure the same split each time.
np.random.shuffle(all_data)
training_data, testing_data = np.split(all_data, [int(0.8 * len(all_data))])

# Release unused memory.
del(all_data)
del(experiment)
del(experiment_metadata)
del(wind_dir)
gc.collect()

0

In [2]:
print(np.std(training_data[:,1]))

358.887464865987


In [3]:
# Inference and model parameters.
H = 0

prior_param_1 = np.array([0.33, 0.86, 1e8, 1e8])
prior_param_2 = np.array([1, 1, 1e8, 1e8])

params_init = prior_param_1.copy()

# Function for calculating the conditional probability for parameters a, b or Q.
@njit
def log_prob_params(params, prior_param_1, prior_param_2, log_lhood, param_select, dist):
    
    if dist == 'gamma':
        prob = (prior_param_1[param_select] - 1)*np.log(params[param_select])-params[param_select]/prior_param_2[param_select]-log_lhood
    elif dist == 'gaussian':
        prob =  -(params[param_select]-prior_param_1[param_select])**2/(2*prior_param_2[param_select]**2)-log_lhood        
    elif dist == 'uniform':
        prob = -log_lhood
    return  prob

# MCMC Sampler for a, b or Q.
@njit
def sample_params(params, prior_param_1, prior_param_2, ss, precalc2, N_data, param_select, dist):

    # Set current and proposed values for a, b, Q and sigma.
    current_params = params
    proposed_params = current_params.copy()
    proposed_params[param_select] =  np.random.normal(current_params[param_select],ss[param_select])
    
    condition = [0,0,0,1]
    
    # Calculating the conditional probability of current and proposed a,b,Q and sigma.
    log_l_curr_params = -sum(precalc2)/(2*current_params[3]**2) - N_data*np.log(np.sqrt(2*np.pi)*current_params[3])*condition[param_select]
    log_l_prop_params = -sum(precalc2)/(2*proposed_params[3]**2) - N_data*np.log(np.sqrt(2*np.pi)*proposed_params[3])*condition[param_select]
    
    log_p_prop_params = log_prob_params(proposed_params, prior_param_1, prior_param_2, log_l_prop_params, param_select, dist)
    log_p_curr_params = log_prob_params(current_params, prior_param_1, prior_param_2, log_l_curr_params, param_select, dist)
        
    # Acceptance criteria.
    if np.random.uniform(low = 0, high = 1) < np.exp(log_p_prop_params - log_p_curr_params):
        return proposed_params, 1
    else:
        return current_params, 0


# Full MCMC sampler.
def sample_process(N_samples, ss, u, H, data, params_init, prior_param_1, prior_param_2):

    # Intialisation. Pre-allocate memory space for the data where relevant.
    params_means = np.empty((N_samples, len(params_init)))    # rows = N_samples, cols = len(params_init).
    params_means[:] = np.NaN
    params_samples = params_means.copy()
    params = params_init
    count = 0
    negysqr = -data[:,2]**2
    piu = np.pi * u
    zdownsqr = -(data[:,3] - H)**2
    zupsqr = -(data[:,3] + H)**2
    N_data = data[:,1].size
    
    accept_tot_0 = 0
    accept_tot_1 = 0
    accept_tot_2 = 0
    accept_tot_3 = 0

    tot_accepted = 0
    for j in range(N_samples):

        count = count + 1
        if (count % 1000 == 0):
            print('Running sample ' + str(count) + '...')    # Print progress every 1000th sample.

        precalc1 = 2 * params[0] * data[:,1]**params[1]
        precalc2 = (params[2] / precalc1 * piu * np.exp(negysqr / precalc1) * (np.exp(zdownsqr / precalc1) + np.exp(zupsqr / precalc1)) - data[:,0])**2
                
        params, accept_0 = sample_params(params,prior_param_1,prior_param_2,ss,precalc2,N_data,0,'uniform') # a
        params, accept_1 = sample_params(params,prior_param_1,prior_param_2,ss,precalc2,N_data,1,'uniform') # b
        params, accept_2 = sample_params(params,prior_param_1,prior_param_2,ss,precalc2,N_data,2,'uniform') # Q 
        params, accept_3 = sample_params(params,prior_param_1,prior_param_2,ss,precalc2,N_data,3, 'uniform') # sigma
        
        accept_tot_0 += accept_0
        accept_tot_1 += accept_1
        accept_tot_2 += accept_2
        accept_tot_3 += accept_3
        
        params_samples[j] = params.T
        params_means[j] = np.mean(params_samples[:count][:], axis = 0)

    print('Sampling complete.')
    print('a Acceptance Rate: ', accept_tot_0/N_samples)
    print('b Acceptance Rate: ', accept_tot_1/N_samples)
    print('Q Acceptance Rate: ', accept_tot_2/N_samples)
    print('sigma Acceptance Rate: ', accept_tot_3/N_samples)
    
    return params_samples, params_means


# Run sampling.
np.random.seed(117)
params_samples, params_means = sample_process(N_samples = 1000, ss =np.array([0.1, 0.1, 1e5,1e7]), u = wind_speed, H = H, data = training_data,
                                        params_init = params_init, prior_param_1 = prior_param_1, prior_param_2 = prior_param_2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in method uniform of numpy.random.mtrand.RandomState object at 0x000001D9869F1540>) found for signature:
 
 >>> uniform(low=Literal[int](0), high=Literal[int](1))
 
There are 4 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba\core\overload_glue.py: Line 131.
    With argument(s): '(low=int64, high=int64)':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   [1m[1m[1mNo implementation of function Function(<intrinsic stub>) found for signature:
    
    >>> stub(int64, int64)
    
   There are 2 candidate implementations:
   [1m  - Of which 2 did not match due to:
     Intrinsic in function 'stub': File: numba\core\overload_glue.py: Line 35.
       With argument

In [ ]:
# Utility function for traceplots.
def traceplots(x, xnames = None, title = None):

    N, d = x.shape
    fig = plt.figure()
    left, tracewidth, histwidth = 0.1, 0.65, 0.15
    bottom, rowheight = 0.1, 0.8/d
    spacing = 0.05

    for i in range(d):
        # Set the location of the trace and histogram viewports,
        # starting with the first dimension from the bottom of the canvas.
        rowbottom = bottom + i * rowheight
        rect_trace = (left, rowbottom, tracewidth, rowheight)
        rect_hist = (left + tracewidth, rowbottom, histwidth, rowheight)

        # First set of trace plot axes.
        if i == 0:
            ax_trace = fig.add_axes(rect_trace)
            ax_trace.plot(x[:,i])
            ax_trace.set_xlabel("Sample Count")
            ax_tr0 = ax_trace

        # Other sets of trace plot axes that share the first trace's x-axis.
        # Make tick labels invisible so they don't clutter up the plot.
        elif i > 0:
            ax_trace = fig.add_axes(rect_trace, sharex=ax_tr0)
            ax_trace.plot(x[:,i])
            plt.setp(ax_trace.get_xticklabels(), visible=False)

        # Title at the top.
        if i == d-1 and title is not None:
            plt.title(title)

        # Trace y-axis labels.
        if xnames is not None:
            ax_trace.set_ylabel(xnames[i])

        # Trace histograms at the right.
        ax_hist = fig.add_axes(rect_hist, sharey=ax_trace)
        ax_hist.hist(x[:,i], orientation='horizontal', bins=50)
        plt.setp(ax_hist.get_xticklabels(), visible=False)
        plt.setp(ax_hist.get_yticklabels(), visible=False)
        xlim = ax_hist.get_xlim()
        ax_hist.set_xlim([xlim[0], 1.1*xlim[1]])

In [ ]:
traceplots(params_samples, xnames = ['a', 'b', 'Q','sigma'], title = 'MCMC samples')
traceplots(params_means, xnames = ['a', 'b', 'Q','sigma'], title = 'MCMC means')

print(params_samples)

In [ ]:
# Gaussian Plume Model for concentration.
def C_func(x,y,z,u,a,b,Q,H):
    C = Q / (2*a*x**b*np.pi*u)*(np.exp(-(y**2)/(2*a*x**b)))*(np.exp(-(z-H)**2/(2*a*x**b))+np.exp(-(z+H)**2/(2*a*x**b)))
    return C

In [ ]:
# Plotting slices of the plume at set Z values using the mean values of the parameters.

params_mean = params_means[-1]
x = np.linspace(0.1, 200, 201)
y = np.linspace(-20, 20, 201)
#z = np.linspace(0, 400, 201)
X,Y = np.meshgrid(x, y)
Z = 10
C = C_func(X, Y, Z, wind_speed, params_mean[0], params_mean[1], params_mean[2], H)
plt.pcolor(X, Y, C, shading = 'auto')
plt.colorbar()
plt.title('Plume concentration at z = ' + str(Z))
plt.xlabel('x')
plt.ylabel('y')
print('Inferred means: a = ', round(params_mean[0], 2), ', b = ', round(params_mean[1], 2), ', Q = ', params_mean[2], ', sigma = ',  params_mean[3], '.', sep = '')

del(x)
del(y)

In [ ]:
# Calculating the RMSE of this new model based on the data
def RMSE_func(params, u, H, data):

    # Gaussian Plume Model for concentration.

    negysqr = -data['y']**2
    piu = np.pi * u
    zdownsqr = -(data['z'] - H)**2
    zupsqr = -(data['z'] + H)**2
    precalc1 = np.array(2 * params[0] * data['x']**params[1])
    precalc2 = (params[2] / precalc1 * piu * np.exp(negysqr / precalc1) * (np.exp(zdownsqr / precalc1) + np.exp(zupsqr / precalc1)) - data['Concentration'])**2

    RMSE = np.sqrt(sum(precalc2)/data.shape[0])
    print('RMSE = ' + str(RMSE))
    return RMSE


RMSE = RMSE_func(params_mean, wind_speed, H, testing_data)

data_range = np.max(testing_data['Concentration']) - np.min(testing_data['Concentration'])

print('Range = ' + str(data_range))

# # saving_samples = pd.DataFrame({'a':abQ_samples[:,0],'b':abQ_samples[:,1],'Q':abQ_samples[:,2]})
# saving_samples = pd.DataFrame(abQ_samples,columns=['a','b','Q'])
# saving_samples.to_csv('samples.csv')

In [ ]:
# Print the values of a, b and Q
def print_vals(params_samples):

    params_samples = np.array(params_samples)
    vals = ['a','b','Q','sigma']
    for i in range(params_samples.shape[1]):
        param_samples = params_samples[:,i]
        param_90 = np.sort(param_samples)[int(np.floor(param_samples.size*0.9))]
        param_50 = np.sort(param_samples)[int(np.floor(param_samples.size*0.5))]
        param_10 = np.sort(param_samples)[int(np.floor(param_samples.size*0.1))]

        print(vals[i] + ' Mean = ' + str(np.mean(params_samples[:,i])) + ', Confidence ' + str([param_10,param_90]))
        
print_vals(params_samples)